Webscraping


In [2]:
from IPython.display import display
import json
import pandas as pd
pd.options.display.max_columns = None # Display all columns of a dataframe
pd.options.display.max_rows = 700
from pprint import pprint
import re
import requests
from bs4 import BeautifulSoup

In [21]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

chromeOptions = webdriver.ChromeOptions()
#it is way to add options

path = 'C:/Users/User/Documents/Github/Films_DM_2023/chromedriver_win32/' #the path
driver = webdriver.Chrome(executable_path=os.path.join(path, 'chromedriver'))

driver.get("https://www.allocine.fr/")
time.sleep(1) # Let the user actually see something!

C:\Users\User\AppData\Local\Temp\ipykernel_18608\4193159110.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=os.path.join(path, 'chromedriver'))


In [22]:
#Accepter les cookies pour pouvoir passer à la suite
cookies = driver.find_element(By.CLASS_NAME, "jad_cmp_paywall_button-cookies").click()
time.sleep(2)


In [23]:
#Aller dans films 
films = driver.find_element(By.XPATH, '//*[@title="Cinéma"]').click()
time.sleep(2)

In [24]:
#Aller dans films 
tous_films = driver.find_element(By.XPATH, '//*[@title="Tous les films"]').click()
time.sleep(2)


In [25]:
#Choisir les genres
action =driver.find_element(By.XPATH,'//*[@id="filter-entity-mobile"]/div/div[1]/select/option[2]').click()

In [29]:
#Avoir feuille par feuille
films =[]
for each_fiche in driver.find_elements(By.CLASS_NAME,'mdl'):
    #titres
    titre=each_fiche.find_element(By.CLASS_NAME, "meta-title-link").text
    #date sortie
    date_sortie= each_fiche.find_element(By.CLASS_NAME, "date").text
    #durée
    #date_sortie= each_quote.find_element(By.CLASS_NAME, "date").text
    #genres
    genres= each_fiche.find_element(By.CLASS_NAME, "xXx").text #faut réussir à mettre plusieurs genres
    films.append({
                "Titre": titre,
                "date_sortie" : date_sortie,
                "genres": genres,
            })
#dataframe
films_data = pd.DataFrame(films)
films_data



,Titre,date_sortie,genres
0,"Super Mario Bros, le film",5 avril 2023,Action
1,John Wick : Chapitre 4,22 mars 2023,Action
2,Everything Everywhere All at Once,31 août 2022,Action
3,Shazam! La Rage des Dieux,29 mars 2023,Action
4,65 - la Terre d'avant,15 mars 2023,Action
5,Avatar : la voie de l'eau,14 décembre 2022,Action
6,John Wick,29 octobre 2014,Action
7,Operation Fortune: Ruse De Guerre,7 avril 2023,Action
8,Ant-Man et la Guêpe : Quantumania,15 février 2023,Action
9,Braquage Final,30 avril 2021,Action
